# Evitement des collisions - Démonstration (Resnet18)

Dans ce notebook, nous allons utiliser le modèle entrainé en situation réelle avec le moteur TensorRT !

In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras

# Création du contexte

In [2]:
import tensorrt as trt

# Construction de la class du logger
class MyLogger(trt.ILogger):
    def __init__(self):
        trt.ILogger.__init__(self)

    def log(self, severity, msg):
        print("%s : %s" %(severity,msg))
        pass

In [3]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

PRECISION = trt.float32

logger = MyLogger()
runtime = trt.Runtime(logger)

with open("meilleur_model_colab/meilleur_modele_colab_FP16.engine", "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())

Severity.INFO : [MemUsageChange] Init CUDA: CPU +198, GPU +0, now: CPU 283, GPU 2090 (MiB)
Severity.INFO : Loaded engine size: 37 MB
Severity.INFO : [MemUsageSnapshot] deserializeCudaEngine begin: CPU 321 MiB, GPU 2165 MiB
Severity.VERBOSE : Using cublas a tactic source
Severity.INFO : [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +158, GPU +246, now: CPU 479, GPU 2411 (MiB)
Severity.VERBOSE : Using cuDNN as a tactic source
Severity.INFO : [MemUsageChange] Init cuDNN: CPU +241, GPU +351, now: CPU 720, GPU 2762 (MiB)
Severity.INFO : [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +0, now: CPU 719, GPU 2762 (MiB)
Severity.VERBOSE : Deserialization required 6168505 microseconds.
Severity.INFO : [MemUsageSnapshot] deserializeCudaEngine end: CPU 719 MiB, GPU 2762 MiB


In [4]:
# Calcul de la taille mémoire requise pour stocker les données d'entrée / sortie dans le GPU
size_input = trt.volume(engine.get_binding_shape(0))* engine.max_batch_size
size_output = trt.volume(engine.get_binding_shape(1))* engine.max_batch_size

# Allocation de mémoire de type "page-locked" sur l'hôte
input_host_mem = cuda.pagelocked_empty(size_input, trt.nptype(PRECISION))
output_host_mem = cuda.pagelocked_empty(size_output, trt.nptype(PRECISION))

# Allocation de mémoire dans la mémoire GPU
input_device_mem = cuda.mem_alloc(input_host_mem.nbytes)
output_device_mem = cuda.mem_alloc(output_host_mem.nbytes)

# Récupère les adresses en GPU des buffers entrées / sorties
bindings = [int(input_device_mem), int(output_device_mem)]

### Création de l'interface d'acquisition

J'ai créé une classe Image personnelle afin d'optimiser le pipeline avec OpenCV :

In [5]:
import traitlets
import threading
import atexit
import numpy as np


class Camera(traitlets.HasTraits):
    type_camera = traitlets.Unicode("CSI")
    capture_device = traitlets.Integer(default_value=0)
    capture_width = traitlets.Integer(default_value=1280)
    capture_height = traitlets.Integer(default_value=720)
    display_width = traitlets.Integer(default_value=640)
    display_height = traitlets.Integer(default_value=480)
    fps = traitlets.Integer(default_value=30)
    flip = traitlets.Integer(default_value=0)
    image = traitlets.Any()
    video_on = traitlets.Bool(default_value=False)
    
    def __init__(self,*args,**kwargs):
        super(Camera, self).__init__(*args, **kwargs)
        self._running = False
        self.image = np.zeros((self.display_height, self.display_width, 3), dtype=np.uint8)
        
        if self.type_camera.find("CSI")>=0:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_CSI(),cv2.CAP_GSTREAMER)
        else:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_USB(),cv2.CAP_GSTREAMER)

        if self.cap.isOpened():
            print("Caméra initialisée")
        else:
            print("Erreur d'ouverture du flux vidéo")
        atexit.register(self.cap.release)
    
    # Lecture d'une frame
    def capture_image(self):
        re, image = self.cap.read()
        if re:
            image_resized = cv2.resize(image,(int(self.display_width),int(self.display_height)))
            return image_resized
        else:
            return self.image
    
    # ON/OFF de la capture vidéo
    def capture_video(self,run=False):
        if run is True:
            self.video_on = True
        else:
            self.video_on = False
    
    # Lecture d'un flux vidéo
    def _capture_video(self):
        while True:
            if not self._running:
                break
            self.image = self.capture_image()

            
    # Détachement de la caméra
    def release(self):
        self.cap.release()

    # Définition du pipeline pour la caméra CSI
    def _gstreamer_pipeline_CSI(self):
        return("nvarguscamerasrc sensor-id=%d ! "
                "video/x-raw(memory:NVMM),"
                "width=(int)%d,height=(int)%d,"
                "format=(string)NV12, framerate=(fraction)%d/1 ! "
                "nvvidconv flip-method=%d ! "
                "video/x-raw,"
                "width=(int)%d,height=(int)%d,"
                "format=(string)BGRx ! videoconvert ! "
                "video/x-raw, format=(string)BGR ! "
                "appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip, self.display_width,self.display_height))

    # Définition du pipeline pour la USB
    def _gstreamer_pipeline_USB(self):
        return("v4l2src device=/dev/video%d ! "
               "video/x-raw, width=(int)%d, height=(int)%d, framerate=(fraction)%d/1 ! "
               "videoflip method=%d ! "
               "videoconvert ! "
               "video/x-raw, format=(string)BGR ! appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip))
    
    # Surveillance de la variable "video_on"
    @traitlets.observe('video_on')
    def _on_running(self, change):
        if change['new'] and not change['old']:
            # not running -> running
            self._running = True
            self.thread = threading.Thread(target=self._capture_video)
            self.thread.start()
        elif change['old'] and not change['new']:
            # running -> not running
            self._running = False
            self.thread.join()

Fonction pour initialiser la caméra :

In [6]:
def InitCamera():
    camera = Camera(type_camera="CSI",capture_device=0,
                capture_width=224,capture_height=224,
                display_width=224,display_height=224,
                fps=30,flip=0)
    return camera

Création de l'interface :

In [7]:
import ipywidgets
import traitlets
from IPython.display import display

try :
    camera.capture_video(run=False)
    camera.release()
    del camera
except NameError:
    pass

# Initialise la caméra
camera = InitCamera()

Caméra initialisée


In [8]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets

from jetbot import bgr8_to_jpeg

# Lance la caméra
camera.capture_video(run=True)

# Création de l'interface
image = widgets.Image(format='jpeg', width=224, height=224)
slider_bloquer = widgets.FloatSlider(description='bloquer', min=0.0, max=1.0, orientation='vertical')
slider_vitesse = widgets.FloatSlider(description='vitesse', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

lien_camera = traitlets.dlink((camera, 'image'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.VBox([widgets.HBox([image, slider_bloquer]), slider_vitesse]))

Il faut ensuite créer une instance du robot pour pouvoir le commander :

In [9]:
from jetbot import Robot

robot = Robot()

Ensuite, on créé une fonction qui va être appellée à chaque fois qu'une nouvelle image est disponible. Cette fonction doit réaliser les opérations suivantes :

1. Pré-traitement de l'image issue de la caméra
2. Appliquer le modèle sur l'image récupérée
3. Si le modèle indique que nous sommes bloqué, alors le robot tourne à gauche. Sinon le robot avance.

In [15]:
import time

n_images = 0
somme_proba = 0.0

def mise_a_jour(change):
    global slider_bloquer, robot
    global n_images
    global somme_proba
    
    x = change['new']                               # (224,224,3)
    #x = traitement_image(x)                        # Aucun traitement ici
    x = x.astype(np.float32)
    x = np.expand_dims(x,axis=0)                          # (1,224,224,3)
    
    np.copyto(input_host_mem,x.ravel())
    
    cfx.push()
    cuda.memcpy_htod(input_device_mem, input_host_mem)
    context.execute(batch_size=1,bindings=bindings)
    cuda.memcpy_dtoh(output_host_mem, output_device_mem)
    cfx.pop()
    
    proba_bloquer = float(np.asarray(output_host_mem[0]))
    
    n_images = n_images + 1
    somme_proba = somme_proba + proba_bloquer

    if n_images == 1:
        proba_moyenne = somme_proba/1
        slider_bloquer.value = proba_moyenne
        if proba_moyenne < 0.5:
            robot.forward(slider_vitesse.value)
        else:
            robot.left(slider_vitesse.value)
        n_images = 0
        somme_proba = 0.0
    
    time.sleep(0.001)

In [16]:
print("Création du context...")
cfx = cuda.Device(0).make_context()
context = engine.create_execution_context()

Création du context...
Severity.INFO : [MemUsageSnapshot] ExecutionContext creation begin: CPU 1773 MiB, GPU 3839 MiB
Severity.VERBOSE : Using cublas a tactic source
Severity.INFO : [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +158, GPU +6, now: CPU 1931, GPU 3846 (MiB)
Severity.VERBOSE : Using cuDNN as a tactic source
Severity.INFO : [MemUsageChange] Init cuDNN: CPU +240, GPU -40, now: CPU 2171, GPU 3806 (MiB)
Severity.VERBOSE : Total per-runner device memory is 29800960
Severity.VERBOSE : Total per-runner host memory is 30016
Severity.VERBOSE : Allocated activation device memory of size 3671040
Severity.INFO : [MemUsageSnapshot] ExecutionContext creation end: CPU 2170 MiB, GPU 3797 MiB
Severity.INFO : [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +0, now: CPU 2170, GPU 3802 (MiB)


In [17]:
mise_a_jour({'new': camera.image})  # Appel la fonction une première fois pour initialiser le réseau

Nous avons donc maintenant notre réseeau de neurones en place, mais il reste à créer le lien entre la caméra et la fonction de mise à jour pour récupérer les images filmées. Cela se fait avec la fonction ``observe``.

In [18]:
camera.observe(mise_a_jour, names='image')

Vous devriez maintenant voir le slider indiquer la probabilité d'être bloqué ou non ! Il suffit d'augmenter la vitesse pour tester le bon fonctionnement du robot en déplacement.

Pour arrêter les expériences, il suffit de rompre le lien avec la fonction de mise à jour :

In [14]:
import time

camera.unobserve(mise_a_jour, names='image')

time.sleep(0.1)  # Petit temps d'attente pour que l'image soit bien traitée par le modèle

robot.stop()

In [ ]:
input_device_mem.free()
output_device_mem.free()
del input_host_mem
del output_host_mem
del context
del engine

Avant de quitter, il faut fermer la caméra :

In [ ]:
try :
    camera.capture_video(run=False)
    camera.release()
    del camera
except NameError:
    pass